# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1. 

### Обязательная часть

Будем парсить страницу со свежеми новостям на [habr.com/ru/all/](https://habr.com/ru/all/).

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

`KEYWORDS = ['python', 'парсинг']`

 Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы). 
 
В итоге должен формироваться датафрейм вида: `<дата> - <заголовок> - <ссылка>`

### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.  

Итоговый датафрейм формировать со столбцами: `<дата> - <заголовок> - <ссылка> - <текст_статьи>`


In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from datetime import datetime
from datetime import timedelta
import time

In [190]:
keywords = ['коронавирус', 'intel', 'sql']
posts_df = pd.DataFrame()

# запрашиваем страницу с превью свежих статей
preview_page = BeautifulSoup(requests.get('https://habr.com/ru/all/').text, 'html.parser')

# выбираем все html-теги, соответствующие статьям
posts = preview_page.find_all('article', class_='post')

for post in posts:
    # получаем ссылку на пост
    title_element = post.find('a', class_='post__title_link')

    # запрашиваем страницу с полной версией статьи
    article_page = BeautifulSoup(requests.get(title_element.attrs.get('href')).text, 'html.parser')
    time.sleep(0.1)

    # выбираем полезное содержимое страницы
    content = article_page.find('article', class_='post_full')
    
    # выполняем поиск по ключевым словам
    for keyword in keywords:
        tags = content.find_all(True, string = re.compile(keyword, flags = re.IGNORECASE))
        if tags:
                # ссылку, название и дату поста берём из мета-данных
                post_date = article_page.find('meta', property="aiturec:datetime")
                post_title = article_page.find('meta', property='og:title')
                post_link = article_page.find('meta', property='og:url')
                post_text = article_page.find('div', class_='post__text')
                row = ({'Дата' : post_date.attrs.get('content')[:10],
                        'Заголовок': post_title.attrs.get('content'), 
                        'Ссылка': post_link.attrs.get('content'),
                        'Текст статьи' : post_text.text})
                posts_df = pd.concat([posts_df, pd.DataFrame([row])])  
                # если найдено хоть одно из ключевых слов, дальше продолжать цикл по keywords не имеет смысла
                break   
                
posts_df = posts_df.reset_index().loc[:, posts_df.reset_index().columns != 'index']
posts_df


,Дата,Заголовок,Ссылка,Текст статьи
0,2021-02-11,Как бросить курить по ВОЗ: с помощью чатбота W...,https://habr.com/ru/post/541980/,Восьмого декабря 2020 года на официальном сайт...
1,2021-02-11,Компьютерное зрение в промышленной дефектоскоп...,https://habr.com/ru/company/itmai/blog/541858/,\nСегодня мы хотим поделиться опытом решения з...
2,2021-02-11,Как совершить транзакцию в Nest.js,https://habr.com/ru/post/541866/,Во множестве случаев разработчики должны испол...
3,2021-02-11,Наглядно о фишинге в 2020 году на примере зоны RU,https://habr.com/ru/company/searchinform/blog/...,Пандемия определённо внесла коррективы во мног...


## Задание 2.

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/).
Список email-ов задаем переменной в начале кода:  
`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: `<почта> - <дата утечки> - <источник утечки> - <описание утечки>`  

**Подсказка**: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [2]:
import json

In [3]:
def check_email(email):
    
    df = pd.DataFrame()
    
    url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
    payload = {'emailAddresses':[email]}
    headers = {
        'Host': 'identityprotection.avast.com',
        'Connection': 'keep-alive',
        'Content-Length': '32',
        'sec-ch-ua': '"Chromium";v="88", "Google Chrome";v="88", ";Not A Brand";v="99"',
        'Accept': 'application/json, text/plain, */*',
        'Vaar-Version': '0',
        'sec-ch-ua-mobile': '?0',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36',
        'Vaar-Header-App-Product': 'hackcheck-web-avast',
        'Content-Type': 'application/json;charset=UTF-8',
        'Origin': 'https://www.avast.com',
        'Sec-Fetch-Site': 'same-site',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Dest': 'empty',
        'Referer': 'https://www.avast.com/',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7'
        }
    
    try:
        res = requests.post(url, data = json.dumps(payload), headers=headers)
        res_dict = json.loads(res.text)
        for key in res_dict['breaches']:
            row = ({'e-mail' : email,
                    'Дата': res_dict['breaches'][key]['publishDate'][:10], 
                    'Источник': res_dict['breaches'][key]['site'],
                    'Описание' : res_dict['breaches'][key]['description']})
            df = pd.concat([df, pd.DataFrame([row])])
    except:
        print('Ошибка, не удалось проверить e-mail {}'.format(email))
    finally:
        return df


In [6]:
emails = ['xxx@ya.ru', 'petrovvv@gmail.com']

leakes = pd.DataFrame()
for email in emails:
    leakes = pd.concat([leakes, check_email(email)])
    time.sleep(0.5)
leakes = leakes.reset_index().loc[:, leakes.reset_index().columns != 'index']
leakes

,e-mail,Дата,Источник,Описание
0,xxx@ya.ru,2016-11-07,ir.netease.com,"In October 2015, the Chinese internet and gami..."
1,xxx@ya.ru,2020-07-30,hurb.com,"In March 2019, the Brazilian travel agency Her..."
2,xxx@ya.ru,2019-03-28,verifications.io,Big data e-mail verification platform verifica...
3,xxx@ya.ru,2020-05-21,vk.com,"At some time in 2020, the Russian social netwo..."
4,xxx@ya.ru,2016-11-01,qip.ru,"In 2011, Russian instant messaging service pro..."
5,xxx@ya.ru,2019-12-05,wowprogress.com,"At an unconfirmed date, the World of Warcraft ..."
6,xxx@ya.ru,2021-01-07,jivo.ru,"At an unconfirmed date, the Russian chat appli..."
7,xxx@ya.ru,2017-11-04,myheritage.com,"In October 2017, a customer database belonging..."
8,xxx@ya.ru,2020-02-06,flyffpenya-shop.com,"In 2015, online gaming shop Flyffpenya Shop wa..."
9,xxx@ya.ru,2020-02-06,cityofvillains.com,"In January 2015, the online role-playing game ..."


### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.  
Документация к API VK: https://vk.com/dev/methods
, вам поможет метод [wall.get](https://vk.com/dev/wall.get)  
```
GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ  
```

В итоге должен формироваться датафрейм со столбцами: `<дата поста> - <текст поста>`

In [192]:
WALLGET_REQUEST = 'https://api.vk.com/method/wall.get?'

params = {
    'access_token': '31585f5cdb2b495be1f1693fcb8fbe9014769b3f8bd93915054b70cd2880a3f4001c0cfffcc76c3b29dbf',
    'v': '5.130',
    'owner_id' : -30159897, # группа Нетологии
    'count': 50
}

res = requests.get(WALLGET_REQUEST, params)

In [193]:
res_dict = json.loads(res.text)

df = pd.DataFrame()
for post in res_dict['response']['items']:
    row = ({'Дата' : datetime.fromtimestamp(post['date']),
            'Сcылка': 'https://vk.com/netology?w=wall-30159897_' + str(post['id']),
            'Текст записи': post['text'],
            'Лайки': post['likes']['count'],
            'Репосты': post['reposts']['count'],
            'Комментарии': post['comments']['count']})
    df = pd.concat([df, pd.DataFrame([row])])
    
df = df.reset_index().loc[:, df.reset_index().columns != 'index']
df.head()

,Дата,Сcылка,Текст записи,Лайки,Репосты,Комментарии
0,2021-02-10 17:13:00,https://vk.com/netology?w=wall-30159897_78295,"Говорят, от даты рождения зависит характер и д...",30,19,26
1,2021-02-11 11:08:00,https://vk.com/netology?w=wall-30159897_78323,"Погоня и перестрелка с полицией в GTA, загадоч...",4,3,0
2,2021-02-10 10:53:00,https://vk.com/netology?w=wall-30159897_78288,📆 15 февраля стартует бесплатный курс «Разрабо...,54,12,0
3,2021-02-09 17:57:00,https://vk.com/netology?w=wall-30159897_78287,"Раз, два, три — эмоция замри 🤪 \n \nЭмоциональ...",137,54,0
4,2021-02-09 10:27:00,https://vk.com/netology?w=wall-30159897_78282,"Кто стоит за всеми приложениями, играми и сайт...",11,3,0


#### ПРИМЕЧАНИЕ
Домашнее задание сдается ссылкой на репозиторий [GitHub](https://github.com/).
Не сможем проверить или помочь, если вы пришлете:
- файлы;
- архивы;
- скриншоты кода.

Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в slack.

##### Как правильно задавать вопросы аспирантам, преподавателям и коллегам?
Прежде чем задать вопрос необходимо попробовать найти ответ самому в интернете. Навык самостоятельного поиска информации – один из важнейших, и каждый практикующий специалист любого уровня это делает каждый день.

Любой вопрос должен быть сформулирован по алгоритму:  
1) Что я делаю?  
2) Какого результата я ожидаю?  
3) Как фактический результат отличается от ожидаемого?  
4) Что я уже попробовал сделать, чтобы исправить проблему?  

По возможности, прикрепляйте к вопросу скриншоты, либо ссылки на код. Оставляйте только проблемный и воспроизводимый участок кода, все решение выкладывать не допускается.
